In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
tokenizer = Tokenizer()

data = open('data.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

#print(tokenizer.word_index)
print(total_words)

11889


In [3]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [4]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150 )))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
earlystop = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=5, verbose=0, mode='auto')

print( model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 100)           1188900   
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               301200    
_________________________________________________________________
dense (Dense)                (None, 11889)             3578589   
Total params: 5,068,689
Trainable params: 5,068,689
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\user\anaconda3\envs\ai\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
model.fit(xs,ys , epochs = 100 , callbacks = [earlystop])

Epoch 1/100
5479/5479 [==============================] - 55s 9ms/step - loss: 6.2362 - accuracy: 0.0852
Epoch 2/100
5479/5479 [==============================] - 53s 10ms/step - loss: 5.4628 - accuracy: 0.1366
Epoch 3/100
5479/5479 [==============================] - 54s 10ms/step - loss: 5.0794 - accuracy: 0.1581
Epoch 4/100
5479/5479 [==============================] - 53s 10ms/step - loss: 4.7687 - accuracy: 0.1753
Epoch 5/100
5479/5479 [==============================] - 54s 10ms/step - loss: 4.4899 - accuracy: 0.1932
Epoch 6/100
5479/5479 [==============================] - 55s 10ms/step - loss: 4.2390 - accuracy: 0.2131
Epoch 7/100
5479/5479 [==============================] - 53s 10ms/step - loss: 4.0118 - accuracy: 0.2361
Epoch 8/100
5479/5479 [==============================] - 53s 10ms/step - loss: 3.8101 - accuracy: 0.2592
Epoch 9/100
5479/5479 [==============================] - 53s 10ms/step - loss: 3.6257 - accuracy: 0.2833
Epoch 10/100
5479/5479 [==============================] 

In [ ]:
model.save("model.h5")

In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model("model.h5")

tokenizer = Tokenizer()

data = open('data.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])

seed_text = "Iron Man  : "
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)
